*Creating A Feedforward Neural Network In PyTorch To Predict Earthquake Depth Ranges*


Import libraries

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np

Load the cleaned csv file.

In [17]:
# Load dataset
clean_eq = pd.read_csv('/Users/gerryjr/Desktop/EarthquakeProject/Data/clean_earthquake.csv')

Before getting into the process of filling/cleaning any missing data, let's create ranges(bins) to categorize earthquake depths. Concurrently, let's map the "labels" which are strings. This gives the strings of "shallow", "intermediate" and "deep" a numeric label of 0, 1 and 2. Since this will be a Feedforward Neural Network, the model needs the inputs to be numeric.

In [18]:
# Create depth ranges
clean_eq['depth_range'] = pd.cut(clean_eq['depth'], bins=[0, 70, 300, float('inf')], labels=['Shallow', 'Intermediate', 'Deep'])

# Map depth ranges to numeric labels
depth_mapping = {'Shallow': 0, 'Intermediate': 1, 'Deep': 2}
clean_eq['depth_range'] = clean_eq['depth_range'].map(depth_mapping)

Next, this drops any rows in the "depth_range" column that has missing or non-numeric values. This will be important for the model.

In [19]:

clean_eq = clean_eq.dropna(subset=['depth_range'])

Just like the TensorFlow FNN, features are engineered. The follow feature engineering is related to depth which will be important for the model to train from.

In [ ]:
#Feature Engineering
clean_eq['depth_squared'] = clean_eq['depth'] ** 2
clean_eq['depth_error_ratio'] = clean_eq['depthError'] / clean_eq['depth']
clean_eq['depth_error_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
clean_eq['depth_error_ratio'].fillna(clean_eq['depth_error_ratio'].median(), inplace=True)
clean_eq['lat_long_interaction'] = clean_eq['latitude'] * clean_eq['longitude']


The features are listed with the fixed and engineered variants. In this case, the target is "depth_range."

In [21]:
# Features and target
X = clean_eq[['depth',
              'gap',
              'rms',
              'depth_squared',
              'latitude',
              'longitude',
              'depth_error_ratio',
              'lat_long_interaction']]
y = clean_eq['depth_range']





Like other models, this splits the testing and training data with 20% being set aside for testing.

In [22]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train = X_train.fillna(X_train.mean())  # or .median()
X_test = X_test.fillna(X_test.mean())


Since this is a neural network, it is important to scale the features.

In [23]:
# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

The below cell implenets SMOTE which stand sfor Synthetic Minority Over-Sampling Technique. This allows the model to deal with class imbalances in the data. In the previous Jupyter Notebook you will notice the distribution plot showed am imbalance of earthquakes of magnitudes ~5. This means that the model can encounter an issue with identifying outliers (aka minorities). I added this cell afterwards when the model was struggling to identify intermidiate and deep depth ranges. This will become more apparent when the results are shown for the model performance.

In [ ]:
# Apply SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

While this looks a bit abstract, these 4 lines simply tell the model that 32-bit floating are the inputs. 32-floating point numbers are standard procedure for deep learning models.

In [25]:
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

Now it is time to create the FNN. Since this is utilizing PyTorch the syntax. Since PyTorch runs the computations dynamically, the code is slightly more rigorous than TensorFlow. However, the overall methodology does not change. A dropout of 40% is added to force the model to learn patterns instead of memorizing the data. The "relu" activation function is added for non-linearity. 

In [26]:
# FNN with dropout
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)  # Dropout to reduce overfitting
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

Initialize the function with the layers and size of said layers.

In [27]:
# Initialize model
input_size = X_train.shape[1]
hidden_size = 128  # Adjusted hidden layer size
output_size = len(depth_mapping)
model = FNN(input_size, hidden_size, output_size)

The loss and optimization are defined. This measures how accurate the predictions are and modifies how the model adjusts the neuron weights during the backpropagation portion of the algorithm. 

In [28]:
# Loss function and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)  # L2 regularization


Now it is time to train the model. The iterations (epochs) are set as 50. This is written as a loop for each iteration. Again PyTorch computes dynamically so there is more flexibility in the structure of the algorithm. 

In [ ]:
# Training the FNN
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Since the model is trained, now it is important to evaluate how well it performed. This prints the accuracy and a classification report. This will discriminate the 3 sub-targets(shallow, intermediate and deep) to evaluate how accurately the model predicted them. To have a visual correlation, a "Confusion Matrix" is generate to show how well the model did.

In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = accuracy_score(y_test, predicted)
    print(f"Test Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, predicted, target_names=['Shallow', 'Intermediate', 'Deep']))

    #Confusion matrix
    cm = confusion_matrix(y_test, predicted)
    print("Confusion Matrix:")
    print(cm)


The above output indicates the model is performing quite well, consistently performing in the low to middle 90%. The "shallow" class performed at 97% which is very strong. The intermidiate class had some missclassifications at 71%. This is somewhat expected since the model is conflating scenarios that are close to the "deep" class. Finally, the "deep" class performed at 83% which is respectable. Overall the model performs well.